In [1]:
### IMPORTS ###
import configparser
import os
import csv
from bs4 import BeautifulSoup
import lxml
import imagej
from string import ascii_uppercase
import scyjava as sj
import pandas as pd
import numpy as np
from pathlib import Path
import xarray as xr
### PARAMETERS ###
# Parameters are stored in a configuration file at [user home/.wbif/autoseg.ini]
# (e.g. on Windows this would be "C:\Users\[username]\.wbif\autoseg.ini").
# These parameters are updated each time the analysis runs, so should always
# display the most recently-input values.

# Getting the path to the configuration file.  "Path.home()" gets the user's
# home folder path.
config_path = os.path.join(Path.home(), ".wbif", "autoseg.ini")

# Creating a new instance of the ConfigParser class, which will handle parameter
# reading and writing
config = configparser.ConfigParser()

# Reading the configuration file at the specified path
config.read(config_path)

# Checking that the config object contains the "defaults" section.  If it
# doesn't we create an empty dict.  The empty dict is there so the following
# steps don't fail.
if "defaults" in config:
    defaults = config["defaults"]
else:
    defaults = {}





# Checking the folder the configuration file will be stored in exists.  If it
# doesn't we create it.
if not os.path.exists(config_path):
    os.makedirs(Path(config_path).parent)

# Writing the updated configuration to file
with open(config_path, "w") as configfile:
    config.write(configfile)


### INITIALISING THE SYSTEM ###
# We now need to start the PyImageJ system running.  This will download a copy
# of ImageJ.  The first time this script runs it can take a couple of minutes;
# however, subsequent runs should get past this step in a matter of seconds.
print("Initialising ImageJ (this may take a couple of minutes first time)")

# Setting the amount of RAM the Java Virtual Machine running ImageJ is allowed
# (e.g. Xmx6g loads 6 GB of RAM)
sj.config.add_option("-Xmx8g")

# Initialising PyImageJ with core ImageJ and the plugins we need.  For this, we
# have the Time_Lapse plugin, which offers an alternative for stack focusing.
# We also import the WEKA plugin.
ij = imagej.init(
    [
        "net.imagej:imagej:2.1.0",
        "net.imagej:imagej-legacy",
        "sc.fiji:Time_Lapse:2.1.1",
        "sc.fiji:Trainable_Segmentation:3.2.34",
        "sc.fiji:TrackMate_:5.1.0"
    ],
    headless=False,
)

# Displaying information about the running ImageJ
print(ij.getApp().getInfo(True))


### MAIN PROCESSING STEPS ###
# The main processing steps have been bundled into a series of functions within
# the utils.py file.  We call the "process_stack" function, which in turn runs
# the relevant functions for stack focussing, normalisation and WEKA.

# Processing Ecad image stack
print("Processing image")



# At this point the analysis is complete
print("Complete")


Initialising ImageJ (this may take a couple of minutes first time)
ImageJ 2.1.0/1.53c; Java 1.8.0_282 [amd64]; 27MB of 7282MB
Processing image
Complete


In [2]:
HyperStackDisplayer = sj.jimport("fiji.plugin.trackmate.visualization.hyperstack.HyperStackDisplayer")
TmXmlReader = sj.jimport("fiji.plugin.trackmate.io.TmXmlReader")
TmXmlWriter = sj.jimport("fiji.plugin.trackmate.io.TmXmlWriter")
Logger = sj.jimport("fiji.plugin.trackmate.Logger")
Settings = sj.jimport( "fiji.plugin.trackmate.Settings")
SelectionModel = sj.jimport("fiji.plugin.trackmate.SelectionModel")
DetectorProvider = sj.jimport( "fiji.plugin.trackmate.providers.DetectorProvider")
TrackerProvider = sj.jimport( "fiji.plugin.trackmate.providers.TrackerProvider")
SpotAnalyzerProvider = sj.jimport( "fiji.plugin.trackmate.providers.SpotAnalyzerProvider")
EdgeAnalyzerProvider = sj.jimport( "fiji.plugin.trackmate.providers.EdgeAnalyzerProvider")
TrackAnalyzerProvider = sj.jimport( "fiji.plugin.trackmate.providers.TrackAnalyzerProvider")
jfile = sj.jimport("java.io.File")
Model = sj.jimport("fiji.plugin.trackmate.Model")
Trackmate = sj.jimport("fiji.plugin.trackmate.TrackMate")
Factory = sj.jimport("fiji.plugin.trackmate.detection.LogDetectorFactory")
LAPUtils = sj.jimport("fiji.plugin.trackmate.tracking.LAPUtils")
SparseLAP = sj.jimport("fiji.plugin.trackmate.tracking.sparselap.SimpleSparseLAPTrackerFactory")
FeatureFilter = sj.jimport("fiji.plugin.trackmate.features.FeatureFilter")
ImagePlus = sj.jimport("ij.ImagePlus")
print("done")

done


In [3]:
from tkinter import filedialog
from tkinter import *
root = Tk()
root.filename =  filedialog.askdirectory(initialdir= "C:\\Users\\jw14251\\OneDrive - University of Bristol\\Documents\\PhD\\Imaging")
directory = root.filename

outputPath = (os.getcwd() + "\\" + directory.rsplit("/", 1)[1])
try:
    os.mkdir(outputPath)
except OSError:
    print ("Creation of the directory %s failed" %outputPath)
else:
    print ("Successfully created the directory %s " % outputPath)

Creation of the directory c:\Users\jw14251\OneDrive - University of Bristol\Documents\Python\Trackmate\210525 Reperfusion lyzdsred cd41 cfms failed


In [4]:
for subdir, dirs, files in os.walk(directory):
    for file in files:        
        filepath = subdir + os.sep + file        
#only take .tif files
        if filepath.endswith("\\red.tif"): 
            xmlPath = (filepath.rsplit(".tif"))[0]
            newXML = open(xmlPath + ".xml", "w")   
            print(filepath)             
            imp = ImagePlus(filepath)
            model = Model()
            model.setLogger(Logger.IJ_LOGGER)
            settings = Settings()
            settings.setFrom(imp)
            settings.detectorFactory = Factory()
                        #################### Change these settings based on trackmate parameters #####################################
            settings.detectorSettings={    
                'DO_SUBPIXEL_LOCALIZATION' : True,
                'RADIUS' : 10.000,
                'TARGET_CHANNEL' : 1,
                'THRESHOLD' : 25.000,
                'DO_MEDIAN_FILTERING' : False,
            }
            # Configure tracker
            settings.trackerFactory = SparseLAP()
            settings.trackerSettings = LAPUtils.getDefaultLAPSettingsMap()
            settings.trackerSettings['LINKING_MAX_DISTANCE'] = 20.0
            settings.trackerSettings['GAP_CLOSING_MAX_DISTANCE'] = 20.0
            # Add ALL the feature analyzers known to TrackMate, via
            # providers. 
            # They offer automatic analyzer detection, so all the 
            # available feature analyzers will be added. 
            spotAnalyzerProvider = SpotAnalyzerProvider()
            for key in spotAnalyzerProvider.getKeys():
                print( key )
                settings.addSpotAnalyzerFactory( spotAnalyzerProvider.getFactory( key ) )
            edgeAnalyzerProvider = EdgeAnalyzerProvider()
            for  key in edgeAnalyzerProvider.getKeys():
                print( key )
                settings.addEdgeAnalyzer( edgeAnalyzerProvider.getFactory( key ) )
            trackAnalyzerProvider = TrackAnalyzerProvider()
            for key in trackAnalyzerProvider.getKeys():
                print( key )
                settings.addTrackAnalyzer( trackAnalyzerProvider.getFactory( key ) )
            # #### track filter for immobile spots
            # filter2 = FeatureFilter('TRACK_DISPLACEMENT', 40, True)
            # settings.addTrackFilter(filter2)
            trackmate = Trackmate(model, settings)
            #process
            ok = trackmate.checkInput()
            if not ok:
                print(str(trackmate.getErrorMessage()))
                continue
            ok = trackmate.process()
            if not ok:
                print(str(trackmate.getErrorMessage()))
                continue
            filter1 = FeatureFilter('TOTAL_INTENSITY', 83833.81, True)
            settings.addTrackFilter(filter1)
            jafile = jfile(xmlPath + ".xml")
            writer = TmXmlWriter(jafile)
            writer.appendModel(model)
            writer.appendSettings(settings)
            writer.writeToFile()

C:/Users/jw14251/OneDrive - University of Bristol/Documents/PhD/Imaging/Clotting/210525 Reperfusion lyzdsred cd41 cfms\Unwounded 1\red.tif
MANUAL_SPOT_COLOR_ANALYZER
Spot descriptive statistics
Spot radius estimator
Spot contrast and SNR
Edge target
Edge mean location
Edge velocity
MANUAL_EDGE_COLOR_ANALYZER
Branching analyzer
Track duration
Track index
Track location
Velocity
TRACK_SPOT_QUALITY
C:/Users/jw14251/OneDrive - University of Bristol/Documents/PhD/Imaging/Clotting/210525 Reperfusion lyzdsred cd41 cfms\Unwounded 2\red.tif
MANUAL_SPOT_COLOR_ANALYZER
Spot descriptive statistics
Spot radius estimator
Spot contrast and SNR
Edge target
Edge mean location
Edge velocity
MANUAL_EDGE_COLOR_ANALYZER
Branching analyzer
Track duration
Track index
Track location
Velocity
TRACK_SPOT_QUALITY
C:/Users/jw14251/OneDrive - University of Bristol/Documents/PhD/Imaging/Clotting/210525 Reperfusion lyzdsred cd41 cfms\Wound 1\red.tif
MANUAL_SPOT_COLOR_ANALYZER
Spot descriptive statistics
Spot radius 